In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
#from keras.preprocessing.sequence.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [8]:
"""
Load data and setup train/test split
~91k train reviews, ~39k test
"""
reviews = pd.read_csv("classifier_data.csv")
reviews.dropna(inplace=True)
train, test = train_test_split(reviews, test_size=0.30, random_state=12)
train_x = train['Review']
train_y = train['Real']
test_x = test['Review']
test_y = test['Real']

In [ ]:
"""
SourceS:
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
https://towardsdatascience.com/tensorflow-2-0-data-transformation-for-text-classification-b86ee2ad8877
https://www.tensorflow.org/tutorials/text/text_classification_rnn
"""

In [68]:
#the reviews are already cleaned, but this will help remove any that is left
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=5000,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, split=' ', char_level=False, oov_token=None,
)
tokenizer.fit_on_texts(list(train_x.values))

In [69]:
x_train_tokens = tokenizer.texts_to_sequences(list(train_x.values))
x_test_tokens = tokenizer.texts_to_sequences(list(test_x.values))

In [70]:
max = 0
for i in x_train_tokens:
  if len(i) > max:
    max = len(i)
print(max)

144


In [77]:
x_train = sequence.pad_sequences(x_train_tokens, maxlen = 100, padding='post')
x_test = sequence.pad_sequences(x_test_tokens, maxlen = 100, padding='post')


In [78]:
x_trai

array([[ 98, 963, 561, ...,   0,   0,   0],
       [344,  35,  34, ...,   0,   0,   0],
       [ 53,  52,   2, ...,   0,   0,   0],
       ...,
       [424,   1,  24, ...,   0,   0,   0],
       [ 60,  66,  17, ...,   0,   0,   0],
       [ 35, 481,   8, ...,   0,   0,   0]], dtype=int32)

In [84]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)


model = Sequential()
model.add(tf.keras.layers.Input(shape=max,))
model.add(Embedding(5000, 64))
model.add(tf.keras.layers.Bidirectional(LSTM(64, activation='tanh', dropout=0.20)))
model.add(Dense(1, activation='relu'))

optimizer = tf.keras.optimizers.Adam()
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [85]:
history = model.fit(x_train, train_y.values, epochs = 100, validation_split=0.15, batch_size=64, callbacks=[callback])

Epoch 1/100
1208/1208 [==============================] - 188s 153ms/step - loss: 0.7048 - accuracy: 0.6114 - val_loss: 0.6115 - val_accuracy: 0.6574
Epoch 2/100
 417/1208 [=========>....................] - ETA: 1:55 - loss: 0.5911 - accuracy: 0.6920

KeyboardInterrupt: ignored